In [16]:
import numpy as np

import torch
import torch.nn as nn

In [29]:
def DDM(mu, sequence_length, sigma=0.5, dt=1, seed=None):
    if seed is not None:
        np.random.seed(seed) 
    num_trajectories = len(mu)
    num_steps = int(sequence_length / dt)
    x = np.zeros((num_trajectories, num_steps + 1))

    for i in range(num_trajectories):
        for t in range(num_steps):
            x[i, t + 1] = x[i, t] + mu[i] * dt + sigma * np.sqrt(dt) * np.random.normal()
    
    p = np.sign(mu)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(p, dtype=torch.long)

mu = np.array([-0.64, -0.16, -0.04, 0.04, 0.16,0.64]) 
x, p = DDM(mu, 50)

In [35]:
z = torch.zeros(x.size(0), 1)
x.shape[0]

6

In [28]:
def DDM(mu, sequence_length, sigma=0.5, dt=1, seed=None):
    if seed is not None:
        np.random.seed(seed) 
    num_trajectories = len(mu)
    num_steps = int(sequence_length / dt)
    x = np.zeros((num_trajectories, num_steps + 1))

    for i in range(num_trajectories):
        for t in range(num_steps):
            x[i, t + 1] = x[i, t] + mu[i] * dt + sigma * np.sqrt(dt) * np.random.normal()
    
    p = np.sign(mu)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(p, dtype=torch.long)

def mse_loss(z, p):
    return ((z - p)**2).mean()
    
class DriftDiffusionRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DriftDiffusionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNNCell(input_size, hidden_size, batch_first=True, nonlinearity='tanh')
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, threshold=5.0):
        h_i = torch.zeros(1, x.shape[0], self.hidden_size)
        for i in range(x.shape[1]):
            h_i = self.RNN_decoder(x[:,i], h_i) 
            z += self.fc(h_i) 
        
        return z, out

def train_model(model, optimizer, mu, epochs=10, batch_size=10, sequence_length=50, save_path = 'model.pth'):
    model.train()
    for epoch in range(epochs):
        signals, p = DDM(mu, sequence_length)
        optimizer.zero_grad()
        z, out = model(signals.unsqueeze(-1))  
        loss = mse_loss(z, p)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    #torch.save(model.state_dict(), save_path)


mu = np.array([-0.64, -0.16, -0.04, 0.04, 0.16,0.64])  

lr=1e-3
model = DriftDiffusionRNN(input_size=1, hidden_size=100, output_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_model(model, optimizer, mu,  epochs=1000)


IndexError: Target -1 is out of bounds.

In [18]:
def evaluate_model(model, data_generator, batch_size=10, sequence_length=50, coherence_levels=[-64, -32, -16, -4, 0, 4, 16, 32, 64]):
    model.eval()  #
    with torch.no_grad(): 
        signals, labels = data_generator(batch_size, sequence_length, coherence_levels)
        outputs, hidden_activities = model(signals.unsqueeze(-1))

        probabilities = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(probabilities, dim=1)

        print(f"Predictions: {predictions}")
        print(f"Labels: {labels}")
        print(f"Hidden Activities Shape: {hidden_activities.shape}") 
        
        return predictions, hidden_activities

predictions, hidden_activities = evaluate_model(model, generate_motion_data)            #K, T, N

Predictions: tensor([0, 1, 1, 1, 1, 0, 0, 0, 0, 1])
Labels: tensor([0, 1, 1, 1, 1, 1, 0, 0, 0, 1])
Hidden Activities Shape: torch.Size([10, 50, 100])
